In [1]:
from kafka import KafkaProducer
import json
import time
import random
from math import cos, pi

In [2]:
def generate_sensor_data():
    timestamp = int(time.time())

    # Simulate IoT sensor data for water quality metrics with realistic patterns
    water_temperature = random.uniform(1, 3) + round(20 + 5 * (1 + 0.5 * (1 + cos((timestamp % 86400) / 86400 * 2 * pi))), 2)
    ph_level = random.uniform(0, 1) + round(7.5 + 0.2 * (1 + cos((timestamp % 86400) / 86400 * 2 * pi)), 2)
    turbidity = round(random.uniform(5, 50), 2)  # Turbidity in NTU (Nephelometric Turbidity Units)
    dissolved_oxygen = round(random.uniform(5, 12), 2)  # Dissolved Oxygen in mg/L

    return {
        "timestamp": timestamp,
        "water_temperature": water_temperature,
        "ph_level": ph_level,
        "turbidity": turbidity,
        "dissolved_oxygen": dissolved_oxygen
    }

In [3]:
# Kafka configuration
kafka_topic = "water_quality"
kafka_bootstrap_servers = ["localhost:9092"] 

producer_number = 2
producers = []
producer_ids = []
# Create Kafka producer
for i in range(producer_number):
    producer_ids.append(str(i))
    producers.append(KafkaProducer(
        bootstrap_servers=kafka_bootstrap_servers,
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )
)
    
print(f"Producing messages to Kafka topic '{kafka_topic}'...")

try:
    while True:
        for i in range(producer_number):
            # Generate sensor data
            sensor_data = generate_sensor_data()
    
            # Publish sensor data to Kafka
            producers[i].send(kafka_topic, value = sensor_data, key = producer_ids[i].encode('utf-8'))
    
            print(f"Sent: {sensor_data} Producer key: {producer_ids[i]}")

            # Wait for 1 second
            time.sleep(1)
except KeyboardInterrupt:
    print("Stopped producing messages.")
finally:
    for producer in producers:
        producer.close()

Producing messages to Kafka topic 'water_quality'...
Sent: {'timestamp': 1741188432, 'water_temperature': 27.939466860669047, 'ph_level': 8.09737497190646, 'turbidity': 40.36, 'dissolved_oxygen': 7.54} Producer key: 0
Sent: {'timestamp': 1741188433, 'water_temperature': 27.519620413118034, 'ph_level': 8.462799391797525, 'turbidity': 13.95, 'dissolved_oxygen': 9.68} Producer key: 1
Sent: {'timestamp': 1741188434, 'water_temperature': 28.66284777645066, 'ph_level': 7.7903697595840775, 'turbidity': 42.4, 'dissolved_oxygen': 7.93} Producer key: 0
Sent: {'timestamp': 1741188435, 'water_temperature': 28.711931146804396, 'ph_level': 8.111508335324771, 'turbidity': 24.95, 'dissolved_oxygen': 6.7} Producer key: 1
Sent: {'timestamp': 1741188436, 'water_temperature': 27.495814640983827, 'ph_level': 8.020012965448155, 'turbidity': 42.03, 'dissolved_oxygen': 8.73} Producer key: 0
Sent: {'timestamp': 1741188437, 'water_temperature': 28.777181866774306, 'ph_level': 7.797788919340578, 'turbidity': 32.